In [1]:
from numpy.core.fromnumeric import mean
from flask import jsonify
import pandas as pd
import numpy as np
import json
import re
import http
import xgboost as xgb
import time
import urllib
from urllib.error import URLError, HTTPError, ContentTooShortError
from datetime import datetime
from itertools import chain, starmap
%load_ext jupyternotify

# years completed 2002-2020
years_arr = [2005, 2006, 2007, 2008, 2009]
def download(url, num_retries=5): 
#     print('Downloading:', url)
    try: 
        
        html = urllib.request.urlopen(url).read()
    except (URLError, HTTPError, ContentTooShortError, http.client.HTTPException, http.client.IncompleteRead) as e: 
        print('Download error:', url)
        html = None 
        if num_retries > 0: 
            if hasattr(e, 'code') and 500 <= e.code < 600: 
                time.sleep(10)
                # recursively retry 5xx HTTP errors 
                return download(url, num_retries - 1) 
        if num_retries > 0: 
            if e == http.client.IncompleteRead: 
                time.sleep(10)
                return download(url, num_retries - 1)
    return html

def flatten_json_iterative(dictionary, sep = '.', ind_start = 0):
    """Flattening a nested json file"""

    def unpack_one(parent_key, parent_value):
        """Unpack one level (only one) of nesting in json file"""
        # Unpacking one level        
        if isinstance(parent_value, dict):
            for key, value in parent_value.items():
                t1 = parent_key + sep + key
                yield t1, value
        elif isinstance(parent_value, list):
            i = ind_start 
            for value in parent_value:
                t2 = parent_key + sep +str(i) 
                i += 1
                yield t2, value
        else:
            yield parent_key, parent_value    

            
    # Continue iterating the unpack_one function until the terminating condition is satisfied
    while True:
        # Continue unpacking the json file until all values are atomic elements (aka neither a dictionary nor a list)
        dictionary = dict(chain.from_iterable(starmap(unpack_one, dictionary.items())))
        # Terminating condition: none of the values in the json file are a dictionary or a list
        if not any(isinstance(value, dict) for value in dictionary.values()) and \
           not any(isinstance(value, list) for value in dictionary.values()):
            break

    return dictionary

def cbb_pbp(gameId):
        """cfb_pbp()
        Pull the game by id
        Data from API endpoints:
        * college-football/playbyplay
        * college-football/summary
        """
        # play by play
        pbp_url = "http://cdn.espn.com/core/mens-college-basketball/playbyplay?gameId={}&xhr=1&render=false&userab=18".format(gameId)
        pbp_resp = download(url=pbp_url)
        pbp_txt = {}
        if pbp_resp is not None:
            pbp_txt['teams'] = np.array([])
            pbp_txt['id'] = np.array([])
            pbp_txt['competitions'] = np.array([])
            pbp_txt['season'] = np.array([])
            pbp_txt['header'] = {}
            pbp_txt['pickcenter'] = np.array([])
            pbp_txt['broadcasts'] = np.array([])
            pbp_txt['videos'] = np.array([])
            pbp_txt['standings'] = np.array([])
            pbp_txt['boxscore'] = np.array([])
            pbp_txt['espnWP'] = np.array([])
            pbp_txt['gameInfo'] = np.array([])
            pbp_txt['season'] = np.array([])
            pbp_txt['timeouts'] = {}
            pbp_d = json.loads(pbp_resp)
            pbp_txt = pbp_d['gamepackageJSON']
            pbp_txt['gameId'] = pbp_d['gameId']
            # summary endpoint for pickcenter array
            summary_url = "http://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/summary?event={}".format(gameId)
            summary_resp = download(url=summary_url)
            summary = json.loads(summary_resp)
            summary_txt = summary['pickcenter']
            # ESPN's win probability
            wp = "winprobability"
            if wp in summary:
                espnWP = summary["winprobability"]
            else:
                espnWP = np.array([])

            if 'news' in pbp_txt.keys():
                del pbp_txt['news']
            if 'shop' in pbp_txt.keys():
                del pbp_txt['shop']
            pbp_txt['gameInfo'] = pbp_txt['header']['competitions'][0]
            pbp_txt['season'] = pbp_txt['header']['season']['year']
            pbp_txt['pickcenter'] = summary_txt
            pbp_txt['espnWP'] = espnWP
            # Home and Away identification variables
            homeTeamId = int(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['id'])
            awayTeamId = int(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['id'])
            homeTeamMascot = str(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['name'])
            awayTeamMascot = str(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['name'])
            homeTeamName = str(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['location'])
            awayTeamName = str(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['location'])
            homeTeamAbbrev = str(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['abbreviation'])
            awayTeamAbbrev = str(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['abbreviation'])
            homeTeamNameAlt = re.sub("Stat(.+)", "St", str(homeTeamName))
            awayTeamNameAlt = re.sub("Stat(.+)", "St", str(awayTeamName))

            if len(pbp_txt['espnWP']) > 1:
                pbp_txt['espnWP'] = espnWP
            else:
                pbp_txt['espnWP'] = espnWP
            pbp_txt['plays_mod'] = []
            for play in pbp_txt['plays']:
                p = flatten_json_iterative(play)
                pbp_txt['plays_mod'].append(p)
            pbp_txt['plays'] = pd.json_normalize(pbp_txt,'plays_mod')
            if len(pbp_txt['plays'])>1:
                pbp_txt['plays']['season'] = pbp_txt['header']['season']['year']
                pbp_txt['plays']['seasonType'] = pbp_txt['header']['season']['type']
                pbp_txt['plays']["awayTeamId"] = awayTeamId
                pbp_txt['plays']["awayTeamName"] = str(awayTeamName)
                pbp_txt['plays']["awayTeamMascot"] = str(awayTeamMascot)
                pbp_txt['plays']["awayTeamAbbrev"] = str(awayTeamAbbrev)
                pbp_txt['plays']["awayTeamNameAlt"] = str(awayTeamNameAlt)
                pbp_txt['plays']["homeTeamId"] = homeTeamId
                pbp_txt['plays']["homeTeamName"] = str(homeTeamName)
                pbp_txt['plays']["homeTeamMascot"] = str(homeTeamMascot)
                pbp_txt['plays']["homeTeamAbbrev"] = str(homeTeamAbbrev)
                pbp_txt['plays']["homeTeamNameAlt"] = str(homeTeamNameAlt)
                # Spread definition
                pbp_txt['plays']["homeTeamSpread"] = 2.5
                if len(pbp_txt['pickcenter']) > 1:
                    if 'spread' in pbp_txt['pickcenter'][1].keys():
                        gameSpread =  pbp_txt['pickcenter'][1]['spread']
                        homeFavorite = pbp_txt['pickcenter'][1]['homeTeamOdds']['favorite']
                    else:
                        gameSpread =  pbp_txt['pickcenter'][0]['spread']
                        homeFavorite = pbp_txt['pickcenter'][0]['homeTeamOdds']['favorite']

                else:
                    gameSpread = 2.5
                    homeFavorite = True
                pbp_txt['plays']["gameSpread"] = abs(gameSpread)
                pbp_txt['plays']["homeTeamSpread"] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['homeTeamSpread'] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['plays']["homeFavorite"] = homeFavorite
                pbp_txt['plays']["gameSpread"] = gameSpread
                pbp_txt['plays']["homeFavorite"] = homeFavorite

                pbp_txt['plays'] = pd.DataFrame(pbp_txt['plays'])
                pbp_txt['plays']['season'] = pbp_txt['header']['season']['year']
                pbp_txt['plays']['seasonType'] = pbp_txt['header']['season']['type']
                pbp_txt['plays']["homeTeamId"] = homeTeamId
                pbp_txt['plays']["awayTeamId"] = awayTeamId
                pbp_txt['plays']["homeTeamName"] = str(homeTeamName)
                pbp_txt['plays']["awayTeamName"] = str(awayTeamName)
                pbp_txt['plays']["homeTeamMascot"] = str(homeTeamMascot)
                pbp_txt['plays']["awayTeamMascot"] = str(awayTeamMascot)
                pbp_txt['plays']["homeTeamAbbrev"] = str(homeTeamAbbrev)
                pbp_txt['plays']["awayTeamAbbrev"] = str(awayTeamAbbrev)
                pbp_txt['plays']["homeTeamNameAlt"] = str(homeTeamNameAlt)
                pbp_txt['plays']["awayTeamNameAlt"] = str(awayTeamNameAlt)
                pbp_txt['plays']['period.number'] = pbp_txt['plays']['period.number'].apply(lambda x: int(x))
                #----- Figuring out Timeouts ---------
                pbp_txt['timeouts'] = {}
                pbp_txt['timeouts'][homeTeamId] = {"1": [], "2": []}
                pbp_txt['timeouts'][awayTeamId] = {"1": [], "2": []}
                pbp_txt['homeTeamSpread'] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['plays']["gameSpread"] = abs(gameSpread)
                pbp_txt['plays']["homeTeamSpread"] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['plays']["homeFavorite"] = homeFavorite
                pbp_txt['plays']["gameSpread"] = gameSpread
                pbp_txt['plays']["homeFavorite"] = homeFavorite

                #----- Time ---------------
                pbp_txt['plays']['clock.mm'] = pbp_txt['plays']['clock.displayValue'].str.split(pat=':')
                pbp_txt['plays'][['clock.minutes','clock.seconds']] = pbp_txt['plays']['clock.mm'].to_list()
                pbp_txt['plays']['half'] = np.where(pbp_txt['plays']['period.number'] <= 2, "1","2")
                pbp_txt['plays']['lag_half'] = pbp_txt['plays']['half'].shift(1)
                pbp_txt['plays']['lead_half'] = pbp_txt['plays']['half'].shift(-1)

                pbp_txt['plays']['game_play_number'] = np.arange(len(pbp_txt['plays']))+1
                pbp_txt['plays']['text'] = pbp_txt['plays']['text'].astype(str)
                pbp_txt['plays']['id'] = pbp_txt['plays']['id'].apply(lambda x: int(x))
                del pbp_txt['plays']['clock.mm']
            else:
                if len(pbp_txt['pickcenter']) > 1:
                    if 'spread' in pbp_txt['pickcenter'][1].keys():
                        gameSpread =  pbp_txt['pickcenter'][1]['spread']
                        homeFavorite = pbp_txt['pickcenter'][1]['homeTeamOdds']['favorite']
                    else:
                        gameSpread =  pbp_txt['pickcenter'][0]['spread']
                        homeFavorite = pbp_txt['pickcenter'][0]['homeTeamOdds']['favorite']

                else:
                    gameSpread = 2.5
                    homeFavorite = True
                pbp_txt['homeTeamSpread'] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
            
            
            pbp_json = {
                "gameId": pbp_txt['gameId'],
                "plays" : pbp_txt['plays'].to_dict(orient='records'),
                "winprobability" : np.array(pbp_txt['winprobability']).tolist(),
                "boxscore" : pbp_txt['boxscore'],
                "header" : pbp_txt['header'],
                "homeTeamSpread" : np.array(pbp_txt['homeTeamSpread']).tolist(),
                "broadcasts" : np.array(pbp_txt['broadcasts']).tolist(),
                "videos" : np.array(pbp_txt['videos']).tolist(),
                "standings" : pbp_txt['standings'],
                "pickcenter" : np.array(pbp_txt['pickcenter']).tolist(),
                "espnWP" : np.array(pbp_txt['espnWP']).tolist(),
                "gameInfo" : np.array(pbp_txt['gameInfo']).tolist(),
                "season" : np.array(pbp_txt['season']).tolist()
            }
            return pbp_json, pbp_json['season']
        else:
            pass
        
    
schedule = pd.read_csv('cbb_games_info_2002_2021.csv', encoding='latin-1')
schedule = schedule.sort_values(by=['season', 'date'], ascending = False)

games = schedule[(schedule['season'].isin(years_arr))&(schedule['status.type.name']=='STATUS_FINAL')].reset_index()['id']
print(f"Number of Games: {len(games)}, first gameId: {games[0]}")
g, y = cbb_pbp(gameId = games[0])
# for key in g.keys():
#     print(f"{key}: {type(g[key])}")    
json.dumps(g,indent=4)
print(g.keys())
print(pd.DataFrame(g['plays']))
y

<IPython.core.display.Javascript object>

2021-02-26 15:07:28,373 [35344] WARNING  py.warnings:109: [JupyterRequire] C:\Users\saiem\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (12,18,62,65,66,72,73) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Number of Games: 26528, first gameId: 290742509
dict_keys(['gameId', 'plays', 'winprobability', 'boxscore', 'header', 'homeTeamSpread', 'broadcasts', 'videos', 'standings', 'pickcenter', 'espnWP', 'gameInfo', 'season'])
     shootingPlay sequenceNumber period.displayValue  period.number  \
0           False              1            1st Half              1   
1           False              2            1st Half              1   
2           False              3            1st Half              1   
3           False              4            1st Half              1   
4           False              5            1st Half              1   
..            ...            ...                 ...            ...   
318         False            319            2nd Half              2   
319          True            320            2nd Half              2   
320          True            321            2nd Half              2   
321          True            322            2nd Half              2   

2009

In [2]:
%%notify
# 2020: 3586,5486 - no header
# 2019: 
# 2018:
# 2017:
# 2016: 3210, 5661
# 2015: 5547
# 2014: 3405, 3454, 3778, 4053, 4161
# 2013: 792, 2651,2732, 4108, 5150,5376
# 2012: 800, 3705, 4046, 5367, 5425, 5571
# 2011: 3556, 4558, 5522, 5523,5605
# 2010: 2358, 3436, 4817
# 2009: 280760096, 283192169, 283202154, 283330021, 283390311, 283652492, 283652084, 283650062, 283650036, 290052736, 290130155
# 2008: 290312344, 280092736, 273550147 273630171 280092736 280100172 280760096 283192169 283202154 283330021
# 2007: 283390311 283652492 283652084 283650062 283650036 290052736 273550147
# 2006: 
# 2005:
# 2004:
# 2003:
# 2002:
i = 22225
for game in games[i:]:
    print(f"Working on game {i+1} of {len(games)}, gameId: {games[i]}")
    if len(str(game))<9:
        i+=1
        continue
    try:
        g, y = cbb_pbp(gameId=game)
    except (TypeError) as e: 
        print("TypeError: yo", e)
        g, y = cbb_pbp(gameId=game)
        
    except (KeyError) as e:
        print("KeyError: yo", e)
        i+=1
        continue
    except (ValueError) as e:
        print("DecodeError: yo", e)
        i+=1
        continue
    fp = "cbb/{}/{}.json".format(y, game)
    with open(fp,'w') as f:
        json.dump(g, f, indent=4, sort_keys=False) 
#     time.sleep(1)        
    i+=1

Working on game 22226 of 26528, gameId: 250542426
Working on game 22227 of 26528, gameId: 250542329
Working on game 22228 of 26528, gameId: 250542272
Working on game 22229 of 26528, gameId: 250542182
Working on game 22230 of 26528, gameId: 250542117
Working on game 22231 of 26528, gameId: 250542006
Working on game 22232 of 26528, gameId: 250540350
Working on game 22233 of 26528, gameId: 250540349
Working on game 22234 of 26528, gameId: 250540325
Working on game 22235 of 26528, gameId: 250540295
Working on game 22236 of 26528, gameId: 250540276
Working on game 22237 of 26528, gameId: 250540256
Working on game 22238 of 26528, gameId: 250540231
Working on game 22239 of 26528, gameId: 250540195
Working on game 22240 of 26528, gameId: 250540161
Working on game 22241 of 26528, gameId: 250540151
Working on game 22242 of 26528, gameId: 250540113
Working on game 22243 of 26528, gameId: 250540107
Working on game 22244 of 26528, gameId: 250530099
Working on game 22245 of 26528, gameId: 250532458


Working on game 22390 of 26528, gameId: 250502113
Working on game 22391 of 26528, gameId: 250502239
Working on game 22392 of 26528, gameId: 250500098
Working on game 22393 of 26528, gameId: 250502710
Working on game 22394 of 26528, gameId: 250502617
Working on game 22395 of 26528, gameId: 250502541
Working on game 22396 of 26528, gameId: 250502050
Working on game 22397 of 26528, gameId: 250500288
Working on game 22398 of 26528, gameId: 250500242
Working on game 22399 of 26528, gameId: 250500145
Working on game 22400 of 26528, gameId: 250500135
Working on game 22401 of 26528, gameId: 250500027
Working on game 22402 of 26528, gameId: 250502737
Working on game 22403 of 26528, gameId: 250502174
Working on game 22404 of 26528, gameId: 250502132
Working on game 22405 of 26528, gameId: 250502619
Working on game 22406 of 26528, gameId: 250502603
Working on game 22407 of 26528, gameId: 250502597
Working on game 22408 of 26528, gameId: 250502514
Working on game 22409 of 26528, gameId: 250502460


Working on game 22554 of 26528, gameId: 250472132
Working on game 22555 of 26528, gameId: 250472754
Working on game 22556 of 26528, gameId: 250472729
Working on game 22557 of 26528, gameId: 250472515
Working on game 22558 of 26528, gameId: 250472514
Working on game 22559 of 26528, gameId: 250472473
Working on game 22560 of 26528, gameId: 250472405
Working on game 22561 of 26528, gameId: 250472363
Working on game 22562 of 26528, gameId: 250472275
Working on game 22563 of 26528, gameId: 250472244
Working on game 22564 of 26528, gameId: 250472199
Working on game 22565 of 26528, gameId: 250472066
Working on game 22566 of 26528, gameId: 250472026
Working on game 22567 of 26528, gameId: 250470399
Working on game 22568 of 26528, gameId: 250470315
Working on game 22569 of 26528, gameId: 250470195
Working on game 22570 of 26528, gameId: 250470189
Working on game 22571 of 26528, gameId: 250470111
Working on game 22572 of 26528, gameId: 250470104
Working on game 22573 of 26528, gameId: 250470057


Working on game 22718 of 26528, gameId: 250430250
Working on game 22719 of 26528, gameId: 250430213
Working on game 22720 of 26528, gameId: 250432649
Working on game 22721 of 26528, gameId: 250432325
Working on game 22722 of 26528, gameId: 250430130
Working on game 22723 of 26528, gameId: 250430066
Working on game 22724 of 26528, gameId: 250432466
Working on game 22725 of 26528, gameId: 250432379
Working on game 22726 of 26528, gameId: 250432352
Working on game 22727 of 26528, gameId: 250432341
Working on game 22728 of 26528, gameId: 250432275
Working on game 22729 of 26528, gameId: 250432199
Working on game 22730 of 26528, gameId: 250432193
Working on game 22731 of 26528, gameId: 250430295
Working on game 22732 of 26528, gameId: 250430231
Working on game 22733 of 26528, gameId: 250430195
Working on game 22734 of 26528, gameId: 250430104
Working on game 22735 of 26528, gameId: 250430068
Working on game 22736 of 26528, gameId: 250430046
Working on game 22737 of 26528, gameId: 250432305


Working on game 22880 of 26528, gameId: 250402649
Working on game 22881 of 26528, gameId: 250402378
Working on game 22882 of 26528, gameId: 250402230
Working on game 22883 of 26528, gameId: 250402193
Working on game 22884 of 26528, gameId: 250402182
Working on game 22885 of 26528, gameId: 250402066
Working on game 22886 of 26528, gameId: 250402050
Working on game 22887 of 26528, gameId: 250400350
Working on game 22888 of 26528, gameId: 250400276
Working on game 22889 of 26528, gameId: 250400261
Working on game 22890 of 26528, gameId: 250400218
Working on game 22891 of 26528, gameId: 250400119
Working on game 22892 of 26528, gameId: 250400111
Working on game 22893 of 26528, gameId: 250400058
Working on game 22894 of 26528, gameId: 250390038
Working on game 22895 of 26528, gameId: 250390096
Working on game 22896 of 26528, gameId: 250390120
Working on game 22897 of 26528, gameId: 250392197
Working on game 22898 of 26528, gameId: 250390242
Working on game 22899 of 26528, gameId: 250390158


...

Working on game 25980 of 26528, gameId: 243322066
Working on game 25981 of 26528, gameId: 243322026
Working on game 25982 of 26528, gameId: 243322006
Working on game 25983 of 26528, gameId: 243320315
Working on game 25984 of 26528, gameId: 243320276
Working on game 25985 of 26528, gameId: 243320256
Working on game 25986 of 26528, gameId: 243320085
Working on game 25987 of 26528, gameId: 243320084
Working on game 25988 of 26528, gameId: 243322458
Working on game 25989 of 26528, gameId: 243322751
Working on game 25990 of 26528, gameId: 243322501
Working on game 25991 of 26528, gameId: 243322473
Working on game 25992 of 26528, gameId: 243322084
Working on game 25993 of 26528, gameId: 243320052
Working on game 25994 of 26528, gameId: 243322169
Working on game 25995 of 26528, gameId: 243322032
Working on game 25996 of 26528, gameId: 243320057
Working on game 25997 of 26528, gameId: 243322579
Working on game 25998 of 26528, gameId: 243322539
Working on game 25999 of 26528, gameId: 243322427


Working on game 26144 of 26528, gameId: 243282217
Working on game 26145 of 26528, gameId: 243280305
Working on game 26146 of 26528, gameId: 243280292
Working on game 26147 of 26528, gameId: 243280093
Working on game 26148 of 26528, gameId: 243282628
Working on game 26149 of 26528, gameId: 243280242
Working on game 26150 of 26528, gameId: 243280158
Working on game 26151 of 26528, gameId: 243280079
Working on game 26152 of 26528, gameId: 243280071
Working on game 26153 of 26528, gameId: 243280127
Working on game 26154 of 26528, gameId: 243280120
Working on game 26155 of 26528, gameId: 243280275
Working on game 26156 of 26528, gameId: 243280235
Working on game 26157 of 26528, gameId: 243282617
Working on game 26158 of 26528, gameId: 243282463
Working on game 26159 of 26528, gameId: 243282377
Working on game 26160 of 26528, gameId: 243282320
Working on game 26161 of 26528, gameId: 243280326
Working on game 26162 of 26528, gameId: 243280288
Working on game 26163 of 26528, gameId: 243280270


Working on game 26306 of 26528, gameId: 243252582
Working on game 26307 of 26528, gameId: 243252115
Working on game 26308 of 26528, gameId: 243252031
Working on game 26309 of 26528, gameId: 243250357
Working on game 26310 of 26528, gameId: 243250270
Working on game 26311 of 26528, gameId: 243250250
Working on game 26312 of 26528, gameId: 243250070
Working on game 26313 of 26528, gameId: 243252507
Working on game 26314 of 26528, gameId: 243252390
Working on game 26315 of 26528, gameId: 243252382
Working on game 26316 of 26528, gameId: 243252368
Working on game 26317 of 26528, gameId: 243252174
Working on game 26318 of 26528, gameId: 243250315
Working on game 26319 of 26528, gameId: 243250301
Working on game 26320 of 26528, gameId: 243250041
Working on game 26321 of 26528, gameId: 243250150
Working on game 26322 of 26528, gameId: 243250221
Working on game 26323 of 26528, gameId: 243252272
Working on game 26324 of 26528, gameId: 243252241
Working on game 26325 of 26528, gameId: 243252230


Working on game 26470 of 26528, gameId: 243222210
Working on game 26471 of 26528, gameId: 243220152
Working on game 26472 of 26528, gameId: 243220130
Working on game 26473 of 26528, gameId: 243222561
Working on game 26474 of 26528, gameId: 243220151
Working on game 26475 of 26528, gameId: 243210012
Working on game 26476 of 26528, gameId: 243210142
Working on game 26477 of 26528, gameId: 243210156
Working on game 26478 of 26528, gameId: 243210269
Working on game 26479 of 26528, gameId: 243210202
Working on game 26480 of 26528, gameId: 243210194
Working on game 26481 of 26528, gameId: 243212507
Working on game 26482 of 26528, gameId: 243210219
Working on game 26483 of 26528, gameId: 243212287
Working on game 26484 of 26528, gameId: 243210225
Working on game 26485 of 26528, gameId: 243212016
Working on game 26486 of 26528, gameId: 243210119
Working on game 26487 of 26528, gameId: 243210213
Working on game 26488 of 26528, gameId: 243212717
Working on game 26489 of 26528, gameId: 243200142


<IPython.core.display.Javascript object>